In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.tree import export_graphviz
from IPython.display import Image  
import pydotplus
from io import StringIO
from sklearn.metrics import mean_squared_error, r2_score
import time
import datetime
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
df=pd.read_csv('AllSensor.csv',names=['Number','Sensor_Address','Time','Date','Temperature','Humidity','Light','Motion','CO2','VDD'])
#df=pd.DataFrame([df],columns=['Number','Sensor Address','Time','Date','Temperature','Humidity','Light','Motion','CO2','VDD'])
weather=pd.read_csv('weather.csv',names=['Number','Temp_weather','Temperature_Max','Temperature_min','Pressure','Humidity_weather','Wind_speed','Sunset_time','Sunrise_time','Weather','Latitude','Longitude','Time','Date'])
#weather.head(1)
result_matrix = pd.DataFrame(columns=['Name','R_Score','Linear_regression_time','Linear_regression_CPU_time',
                                      'NN','Neural_Network_time','Neural_Network_CPU_time','DT',
                                      'Decision_Tree_time','Decision_Tree_CPU_time','Count'])

In [3]:
m = {}
for i in range(0,61,1):
    if (i >= 0 and i <= 15):
        m[i] = 15
    elif (i>15 and i<= 30):
        m[i] = 30
    elif (i>30 and i<=45):
        m[i] = 45
    else:
        m[i] = 60
s= {
    'a8-17-58-ff-fe-03-64-cb',
    'a8-17-58-ff-fe-03-65-48',
    'a8-17-58-ff-fe-03-65-49',
    'a8-17-58-ff-fe-03-65-4a',
    'a8-17-58-ff-fe-03-65-4b',
    'a8-17-58-ff-fe-03-65-4c',
    'a8-17-58-ff-fe-03-65-4d',
    'a8-17-58-ff-fe-03-66-ac',
    'a8-17-58-ff-fe-03-66-af',
    'a8-17-58-ff-fe-03-66-b6',
    'a8-17-58-ff-fe-03-66-b7',
    'a8-17-58-ff-fe-03-66-b8',
    'a8-17-58-ff-fe-03-66-b9',
    'a8-17-58-ff-fe-03-66-ba',
    'a8-17-58-ff-fe-03-67-1a',
    'a8-17-58-ff-fe-03-93-8e',
    'a8-17-58-ff-fe-03-93-8f',
    'a8-17-58-ff-fe-03-93-90',
    'a8-17-58-ff-fe-03-93-91',
    'a8-17-58-ff-fe-03-93-92',
    'a8-17-58-ff-fe-03-93-93',
    'a8-17-58-ff-fe-03-93-97',
    'a8-17-58-ff-fe-03-93-d0',
    'a8-17-58-ff-fe-03-93-d1',
    'a8-17-58-ff-fe-03-93-d2',
    'a8-17-58-ff-fe-03-93-d3',
    'a8-17-58-ff-fe-03-93-d4',
    'a8-17-58-ff-fe-03-93-d7'
    #'a8-17-58-ff-fe-03-93-d8',
    'a8-17-58-ff-fe-03-93-d9',
    'a8-17-58-ff-fe-03-93-da'}
W={'Clear':1,'Clouds':2,'Drizzle':3,'Fog':4,'Haze':5,'Mist':6,'Rain':7,'Snow':8,'Thunderstorm':9}

In [4]:
weather = weather.drop(['Number','Latitude','Longitude'],axis = 1)
weather['DateTime'] = weather['Date'] + ' ' + weather['Time']
weather['Time'] = pd.to_datetime(weather['Time'], format='%H:%M:%S').dt.time
weather['Date'] = pd.to_datetime(weather['Date'], format='%Y-%m-%d').dt.date
weather['DateTime'] = pd.to_datetime(weather['DateTime'], format='%Y-%m-%d %H:%M:%S')
weather['Minute']=weather.DateTime.dt.minute
weather['Hour']=weather.DateTime.dt.hour
weather.Weather = [W[i] for i in weather.Weather]
weather.Minute = weather.Minute.replace(m)
weather['sunset']= pd.to_datetime(weather['Sunset_time'],format='%H:%M:%S')
weather['sunset'] = (weather['sunset'] - datetime.datetime(1900, 1, 1)).dt.total_seconds()
#weather.sunset = weather.sunset.dt.total_seconds()
weather['sunrise']= pd.to_datetime(weather['Sunrise_time'],format='%H:%M:%S')
weather['sunrise'] = (weather['sunrise'] - datetime.datetime(1900, 1, 1)).dt.total_seconds()
weather['Month'] = weather.DateTime.dt.month
weather['Day'] = weather.DateTime.dt.day
weather['Year'] = weather.DateTime.dt.year
weather = weather.groupby(['Month','Day','Hour','Minute'],as_index = False).mean()
weather = weather.round(2)
weather.head(1)

,Month,Day,Hour,Minute,Temp_weather,Temperature_Max,Temperature_min,Pressure,Humidity_weather,Wind_speed,Weather,sunset,sunrise,Year
0,3,3,17,60,6.58,8.73,3.6,1003.4,67.0,7.3,2.0,64004.0,24391.0,2020.0


In [5]:
df.dropna
df['DateTime'] = df['Date'] + ' ' + df['Time']
df = df.drop(['Number','Date','Time'], axis =1)
#df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S',errors='coerce').dt.time
#df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d',errors='coerce').dt.date
df.dropna
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%Y-%m-%d %H:%M:%S',errors='coerce')
df.dropna
df['Minute']=df.DateTime.dt.minute
df.Minute = df.Minute.replace(m)
df['Hour']=df.DateTime.dt.hour
df['Day']=df.DateTime.dt.day
df['Month']=df.DateTime.dt.month
df['Year']=df.DateTime.dt.year
df = df.round(2)
df.head(1)

,Sensor_Address,Temperature,Humidity,Light,Motion,CO2,VDD,DateTime,Minute,Hour,Day,Month,Year
0,a8-17-58-ff-fe-03-93-d4,22.6,33.0,2.0,0.0,711.0,3.63,2020-02-14 15:40:44,45.0,15.0,14.0,2.0,2020.0


In [6]:
for i in s:
    try:
        df1 = df[df['Sensor_Address']== i ]
        df1.dropna
        df1 = df1.groupby(['Month','Day','Hour','Minute'],as_index = False).mean()
        result = pd.merge(weather, df1, on = ['Month','Day','Hour','Minute','Year'])
        count = len(result)
        X = pd.DataFrame(result,columns=['Temp_weather','Pressure','Humidity_weather','Wind_speed','Weather','Month','Day','Hour','Minute','sunset','sunrise'])
        y = result['Temperature'].round().astype(int)
        X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)
        lr = time.time()
        clr = time.clock()
        Model = LinearRegression()
        Model.fit(X_train, y_train)
        lr = time.time() - lr
        clr = time.clock() - clr
        Model.score(X_train, y_train)
        y_pred = Model.predict(X_test)
        rscore = r2_score(y_test, y_pred)
        print ('rscore is', rscore)
        dt = time.time()
        cdt = time.clock()
        clf = DecisionTreeClassifier()
        # Train Decision Tree Classifer
        clf = clf.fit(X_train,y_train)
        dt = time.time() - dt
        cdt = time.clock() - cdt
        y_pred = clf.predict(X_test)
        DT = metrics.accuracy_score(y_test, y_pred)
        print ('DT accuracy', DT)
        scaler = StandardScaler()
        scaler.fit(X_train)
        StandardScaler(copy=True, with_mean=True, with_std=True)
        X_train1 = scaler.transform(X_train)
        X_test1 = scaler.transform(X_test)
        nn = time.time()
        cnn = time.clock()
        mlp = MLPClassifier(hidden_layer_sizes=(11,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,22,11), activation='relu', solver='adam', max_iter=500)
        mlp.fit(X_train1,y_train)
        nn = time.time() - nn
        cnn = time.clock() - cnn
        predictions = mlp.predict(X_test1)
        NN = accuracy_score(y_test, predictions)
        print ('NN accuracy', NN)
        result_matrix = result_matrix.append({'Name': i,'R_Score':rscore,'Linear_regression_time':lr,
                                                  'Linear_regression_CPU_time':clr,
                                                  'NN':NN,'Neural_Network_time':nn,'Neural_Network_CPU_time':cnn,
                                                  'DT':DT,'Decision_Tree_time':dt,'Decision_Tree_CPU_time':cdt,'Count':count
                                                 }, ignore_index=True)
        result = pd.DataFrame(None)
        df1 = pd.DataFrame(None)
    except:
        result = pd.DataFrame(None)
        df1 = pd.DataFrame(None)

rscore is 0.6655968496260645
DT accuracy 0.834400341442595
NN accuracy 0.7323943661971831
rscore is 0.307534104239109
DT accuracy 0.9044890162368673
NN accuracy 0.6685768863419294
rscore is 0.8218544876830167
DT accuracy 0.8760445682451253
NN accuracy 0.5
rscore is 0.5572908280810136
DT accuracy 0.9024509803921569
NN accuracy 0.6877450980392157
rscore is 0.3477560380717397
DT accuracy 0.7890489913544668
NN accuracy 0.768299711815562
rscore is 0.6055625930754783
DT accuracy 0.8843904633378318
NN accuracy 0.8340080971659919
rscore is 0.4791522176495794
DT accuracy 0.8195899772209567
NN accuracy 0.606378132118451
rscore is 0.6305082913387678
DT accuracy 0.9389830508474576
NN accuracy 0.8885593220338983
rscore is 0.4542954581672264
DT accuracy 0.8611111111111112
NN accuracy 0.7795698924731183
rscore is 0.4352447291288537
DT accuracy 0.8850931677018633
NN accuracy 0.7453416149068323
rscore is 0.7891171849927143
DT accuracy 0.931981981981982
NN accuracy 0.8022522522522523
rscore is 0.8364759

In [7]:
result_matrix

,Name,R_Score,Linear_regression_time,Linear_regression_CPU_time,NN,Neural_Network_time,Neural_Network_CPU_time,DT,Decision_Tree_time,Decision_Tree_CPU_time,Count
0,a8-17-58-ff-fe-03-65-4b,0.665597,0.003785,0.005114,0.732394,33.661648,33.645800,0.834400,0.047849,0.063512,7810
1,a8-17-58-ff-fe-03-64-cb,0.307534,0.003952,0.005207,0.668577,10.647160,10.682100,0.904489,0.041128,0.082455,6980
2,a8-17-58-ff-fe-03-93-97,0.821854,0.003062,0.003886,0.500000,4.482954,4.482880,0.876045,0.120330,0.101921,2391
3,a8-17-58-ff-fe-03-65-4a,0.557291,0.003297,0.004345,0.687745,17.442750,17.441510,0.902451,0.227021,0.324145,6797
4,a8-17-58-ff-fe-03-93-90,0.347756,0.003649,0.004725,0.768300,25.758869,25.756101,0.789049,0.232645,0.322123,5782
5,a8-17-58-ff-fe-03-66-ac,0.605563,0.004070,0.005319,0.834008,25.625113,25.622719,0.884390,0.240128,0.354242,7410
6,a8-17-58-ff-fe-03-66-af,0.479152,0.003738,0.004928,0.606378,21.683409,21.681507,0.819590,0.145914,0.237679,7314
7,a8-17-58-ff-fe-03-93-d2,0.630508,0.003470,0.004664,0.888559,21.795563,21.793053,0.938983,0.141936,0.196147,7865
8,a8-17-58-ff-fe-03-93-d0,0.454295,0.003403,0.004658,0.779570,26.269907,26.267949,0.861111,0.144758,0.180198,7438
9,a8-17-58-ff-fe-03-65-4c,0.435245,0.003481,0.004576,0.745342,26.297590,26.294894,0.885093,0.137449,0.102774,7512


for i in s:
    df1 = df[df['Sensor_Address']== i ]
    df1.dropna
    #df1['DateTime'] = df1['Date'] + ' ' + df1['Time']
    #df1 = df1.drop(['Number'], axis =1)
    #df1['Time'] = pd.to_datetime(df1['Time'], format='%H:%M:%S',errors='coerce').dt.time
    #df1['Date'] = pd.to_datetime(df1['Date'], format='%Y-%m-%d',errors='coerce').dt.date
    #df1.dropna
    #df1['DateTime'] = pd.to_datetime(df1['DateTime'], format='%Y-%m-%d %H:%M:%S',errors='coerce')
    #df1.dropna
    #df1['Minute']=df1.DateTime.dt.minute
    #df1['Hour']=df1.DateTime.dt.hour
    result = pd.merge(weather, df1, on=['Date','Hour'])
    result = result.reset_index()
    X = pd.DataFrame(result,columns=['Temp_weather','Pressure','Humidity_weather','Wind_speed','Weather','Hour'])
    y = result['Temperature'].round().astype(int)
    X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)
    lr = time.time()
    clr = time.clock()
    Model = LinearRegression()
    Model.fit(X_train, y_train)
    lr = time.time() - lr
    clr = time.clock() - clr
    Model.score(X_train, y_train)
    y_pred = Model.predict(X_test)
    rscore = r2_score(y_test, y_pred)
    print ('rscore is', rscore)
    scaler = StandardScaler()
    scaler.fit(X_train)
    StandardScaler(copy=True, with_mean=True, with_std=True)
    X_train1 = scaler.transform(X_train)
    X_test1 = scaler.transform(X_test)
    nn = time.time()
    cnn = time.clock()
    mlp = MLPClassifier(hidden_layer_sizes=(6,12,12,12,12,12,12,12,12,12,12,12,12,6), activation='relu', solver='adam', max_iter=500)
    mlp.fit(X_train1,y_train)
    nn = time.time() - nn
    cnn = time.clock() - cnn
    predictions = mlp.predict(X_test1)
    NN = accuracy_score(y_test, predictions)
    print ('NN accuracy', NN)
    dt = time.time()
    cdt = time.clock()
    clf = DecisionTreeClassifier()
    # Train Decision Tree Classifer
    clf = clf.fit(X_train,y_train)
    dt = time.time() - dt
    cdt = time.clock() - cdt
    y_pred = clf.predict(X_test)
    DT = metrics.accuracy_score(y_test, y_pred)
    print ('DT accuracy', DT)
    result_matrix = result_matrix.append({'Name': i,'R_Score':rscore,'Linear_regression_time':lr,
                                          'Linear_regression_CPU_time':clr,
                                          'NN':NN,'Neural_Network_time':nn,'Neural_Network_CPU_time':cnn,
                                          'DT':DT,'Decision_Tree_time':dt,'Decision_Tree_CPU_time':cdt
                                         }, ignore_index=True)
    print (result_matrix)
    result = pd.DataFrame(None)
    df1 = pd.DataFrame(None)
    


feature_cols = ['Temp_weather','Pressure','Humidity_weather','Wind_speed','Weather','Hour']

dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, feature_names = feature_cols,class_names=['20','21','22','23','24','25','26','27','28','29'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('diabetes.png')
Image(graph.create_png())


clf1 = DecisionTreeClassifier(criterion="entropy", max_depth=3)

# Train Decision Tree Classifer
clf1 = clf1.fit(X_train,y_train)

#Predict the response for test dataset
y_pred1 = clf1.predict(X_test)

# Model Accuracy, how often is the classifier correct?
metrics.accuracy_score(y_test, y_pred1)

dot_data = StringIO()
export_graphviz(clf1, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True, feature_names = feature_cols,class_names=['20','21','22','23','24','25','26','27','28','29'])
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('tree.png')
Image(graph.create_png())

In [8]:
#df3 = df2.groupby(['Date'])
#df3 = df3['Temperature'].mean()
#df3 = df3.reset_index()

In [9]:
#plt.scatter(df3['Date'],df3['Temperature'])

In [10]:
#weather1 = weather.groupby(['Date','Hour'])
#weather1 = weather1['Temperature','Temperature_Max','Temperature_min','Pressure','Humidity_weather','Wind_speed'].mean()

In [11]:
#weather1 = weather1.reset_index()

 Name   R_Score        NN        DT
0   a8-17-58-ff-fe-03-66-ac  0.595430  0.744994  0.943777
1   a8-17-58-ff-fe-03-65-4c  0.468339  0.694472  0.954318
2   a8-17-58-ff-fe-03-64-cb  0.146548  0.713001  0.954878
3   a8-17-58-ff-fe-03-66-ba  0.200371  0.683995  0.953583
4   a8-17-58-ff-fe-03-93-90  0.297189  0.706077  0.864526
5   a8-17-58-ff-fe-03-93-92  0.447549  0.919539  0.985631
6   a8-17-58-ff-fe-03-93-97  0.602336  0.754309  0.952748
7   a8-17-58-ff-fe-03-93-d2  0.499422  0.712141  0.963377
8   a8-17-58-ff-fe-03-93-d4  0.545130  0.722693  0.936247
9   a8-17-58-ff-fe-03-93-d9  0.507864  0.711767  0.950648
10  a8-17-58-ff-fe-03-66-af  0.360372  0.594911  0.909053
11  a8-17-58-ff-fe-03-66-b6  0.216660  0.825937  0.869008
12  a8-17-58-ff-fe-03-66-b8  0.520437  0.752808  0.972641
13  a8-17-58-ff-fe-03-65-4b  0.658628  0.666369  0.921631

In [12]:
#result = pd.merge(weather, df1, on=['Date','Time','DateTime','Minute','Hour'])

In [13]:
#result.head()